In [ ]:
import torch
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)


In [ ]:
batch_size = 64  # You can adjust this as per your computational resources

train_loader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)


In [ ]:
class FashionClassifier(nn.Module):
    def __init__(self):
        super(FashionClassifier, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = FashionClassifier()


In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

epochs = 5  # You can adjust this number
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_loader, model, loss_fn, optimizer)
print("Done Training!")


Epoch 1
-------------------------------
loss: 2.300499  [    0/60000]
loss: 0.571715  [ 6400/60000]
loss: 0.551075  [12800/60000]
loss: 0.516063  [19200/60000]
loss: 0.329042  [25600/60000]
loss: 0.412347  [32000/60000]
loss: 0.319959  [38400/60000]
loss: 0.634995  [44800/60000]
loss: 0.806925  [51200/60000]
loss: 0.407362  [57600/60000]
Epoch 2
-------------------------------
loss: 0.382300  [    0/60000]
loss: 0.410304  [ 6400/60000]
loss: 0.277542  [12800/60000]
loss: 0.373559  [19200/60000]
loss: 0.169156  [25600/60000]
loss: 0.185650  [32000/60000]
loss: 0.195986  [38400/60000]
loss: 0.328994  [44800/60000]
loss: 0.312892  [51200/60000]
loss: 0.324507  [57600/60000]
Epoch 3
-------------------------------
loss: 0.485057  [    0/60000]
loss: 0.448124  [ 6400/60000]
loss: 0.363440  [12800/60000]
loss: 0.287689  [19200/60000]
loss: 0.337441  [25600/60000]
loss: 0.231099  [32000/60000]
loss: 0.250052  [38400/60000]
loss: 0.314951  [44800/60000]
loss: 0.247732  [51200/60000]
loss: 0.22

In [ ]:
# Replace 'model' with your model instance
model_scripted = torch.jit.script(model)
model_scripted.save("fashion_model.pt")


# New Section